3. Создание нейронной сети при помощи пакета torch. Dataset и Dataloader.

1. Автоматическое дифференцирование в torch
1.1 Воспользовавшись классами Neuron и MSELoss из задачи 2.4.1 и автоматическим дифференцированием, которое предоставляет torch, решить задачу регрессии. Для оптимизации использовать стохастический градиетный спуск.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression


torch.manual_seed(63)

In [24]:
X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
y = torch.from_numpy(y).to(dtype=torch.float32)

In [32]:
X[:5], X.shape

(tensor([[ 0.6128,  0.5975, -0.5655,  0.7449],
         [ 1.0168, -1.8511,  0.4628,  1.0567],
         [ 0.3447,  2.0135,  0.9885,  2.8184],
         [ 1.2066, -1.3950, -1.5007,  1.2864],
         [-0.0256,  1.1211, -0.3713, -1.1509]]),
 torch.Size([100, 4]))

In [33]:
y[:5], y.shape

(tensor([23.0544, 20.6854, 61.3843,  7.6888,  3.3967]), torch.Size([100]))

In [ ]:
class Neuron:
    def __init__(self, n_inputs):
        # <создать атрибуты объекта weights и bias>
        self.weights = torch.randn(n_inputs, requires_grad=True)
        self.bias = torch.randn(1, requires_grad=True)

    def forward(self, inputs):
        return torch.matmul(inputs, self.weights.T) + self.bias

    def backward(self, dvalue):
        self.dweights = dvalue
        self.dinput = dvalue * self.weights
        self.dbias = dvalue

        return self.dweights, self.dbias

    def parameters(self):
        return self.weights, self.bias


class MSELoss:
    def forward(self, y_pred, y_true):
        diff = y_pred - y_true
        differences_squared = diff ** 2
        mean_diff = differences_squared.mean()

        return mean_diff

    def backward(self, y_pred, y_true):
        diff = y_pred - y_true
        differences_doubled = 2 * diff
        differences_div = differences_doubled / y_pred.size(0)
        return differences_div

In [46]:
neuron = Neuron(X.shape[1])
mse = MSELoss()
optimizer = torch.optim.SGD(neuron.parameters(), lr=0.1)

In [47]:
for epoch in range(16):
    y_pred = neuron.forward(X)
    loss = mse.forward(y_pred=y_pred, y_true=y)
    print('Loss: ', loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

neuron.weights, coef

Loss:  1254.1639404296875
Loss:  814.9367065429688
Loss:  536.2923583984375
Loss:  357.29913330078125
Loss:  240.8580322265625
Loss:  164.15567016601562
Loss:  113.01248168945312
Loss:  78.51460266113281
Loss:  54.99150085449219
Loss:  38.79158401489258
Loss:  27.534318923950195
Loss:  19.64873504638672
Loss:  14.085886001586914
Loss:  10.137386322021484
Loss:  7.319832801818848
Loss:  5.3001251220703125


(tensor([30.2505, 10.6562, 12.0470,  4.5615], requires_grad=True),
 array([31.49371987, 12.20019395, 13.02041604,  4.46822917]))

1.2 Воспользовавшись классами Linear и MSELoss из задачи 2.1.4 и 2.3.1, ReLU из 2.2.1 и автоматическим дифференцированием,
которое предоставляет torch, решить задачу регрессии. Для оптимизации использовать пакетный градиентный спуск.
Вывести график функции потерь в зависимости от номера эпохи. Вывести на одном графике исходные данные и предсказанные
значения.

In [ ]:
class Linear:
  def __init__(self, n_features, n_neurons):
    self.weights = torch.randn(n_features, n_neurons, requires_grad=True)
    self.biases = torch.randn(n_neurons, requires_grad=True)

  def forward(self, inputs: torch.Tensor) -> torch.Tensor:
    return (inputs @ self.weights) + self.biases


class MSELoss:
    def forward(self, y_pred, y_true):
        diff = y_pred - y_true
        differences_squared = diff ** 2
        mean_diff = differences_squared.mean()

        return mean_diff

    def backward(self, y_pred, y_true):
        diff = y_pred - y_true
        differences_doubled = 2 * diff
        differences_div = differences_doubled / y_pred.size(0)
        return differences_div


class ReLU:
  def forward(self, inputs: torch.Tensor) -> torch.Tensor:
    return torch.where(inputs>0, inputs, 0)

In [ ]:
l1 = Linear(X.shape[1], 100)
relu = ReLU()
l2 = Linear(100, y.size(1))
mse = MSELoss()

lr = 0.01
epochs = 1000